In [1]:
#import libraries
import numpy as np
import pandas as pd
import xlsxwriter
import re
import os.path

## 1. Space Remover

In [ ]:
# Delete spaces
def space_remover(dataframe):
    l = []
    for header in dataframe.columns:
        l.append(header.strip().replace('\n',""))
    dataframe.columns = l
    return dataframe

In [ ]:
#Test case
daf = pd.DataFrame({
    'Name': [' John ', 'Mary\n', 'Tom '],
    'Age': [25, 30, 35],
    'Gender': ['Male', 'Female', 'Male']
})

# print the original dataframe
print('Original DataFrame:\n', daf)

# apply the space_remover() function to the dataframe
new = space_remover(daf)

# print the modified dataframe
print('Modified DataFrame:\n', new)

## 2. Cut and Paste (for copy, change column name instead of delete)

In [ ]:
#Parameter: df = the spreadsheet, copy_col = as string (header of col to be copied), local_col = as string (header of col to paste the column in front of)
def cut_paste(df, copy_col,local_col):
    df.rename(columns={copy_col:f'New {copy_col}'}, inplace=True)
    idx = df.columns.get_loc(local_col)
    df.insert(idx, copy_col, df[f'New {copy_col}'])
    del df[f'New {copy_col}']

In [1]:
#Add test case and update

## 3. How to Create Tables (Pivot - Count Items of a Column)

In [ ]:
#Parameter: df = dataframe (can be a spreadsheet), col_count = as string (col header use for counting), col_display = as string (col header which the count is for)

def pivot(df, col_count, col_display):
    pd.pivot_table(df, values= col_count,index= col_display,
               aggfunc='count')


In [2]:
#Add test case
#Add save method such that function will output a spreadsheet contain pivot table
#create function such that can create multiple pivot tables

## 4. Create Dictionary with Count of Sheet to Check

#### Function

In [ ]:
#This funtion will be call after changes is made to the dataframe for checking purposes
#Task: Get the original count -> need to set an original df before run
#      Get the count after changes -> use the current df
#      Compare the two count
#      Output message: Item_name have the same (or difference of) count: Original(orig_num) vs New(new_num)
#Add parameter so that can choose between counting for rows or columns

#### Orignal Code

#### For row, use shape[0]

In [ ]:
#get count of column header before start
counts_OG = dict()
for name, sheet in OG_data.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        counts_OG[name] = counts_OG.get(name, OG_data[name].shape[1])

In [ ]:
counts_OG

In [ ]:
#get count of header after reorder
counts = dict()
for name, sheet in df.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        counts[name] = counts.get(name, df[name].shape[1])

In [ ]:
#Check if values match
for key, value in counts_OG.items():
    if counts_OG[key] != counts[key]:
        print(f"{key} have different value, OG: {counts_OG[key]} vs New: {counts[key]}")

## 5. Check if value of column is different -> can be combine with Func 4

#### Orignal Code

In [ ]:
for name, sheet in df.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        test = df[name]['Col1'] == df[name]['OG Col1']
        row_num = df[name].shape[0]

        count = 0

        for i in range(0, row_num):
            if test[i] == False:
                count = count + 1
        if count != 0:
            print(f'{name} has a differences of {count}')

In [ ]:
#quick check
name = 'Sheet1'
test = df[name]['Last Name'] == df[name]['OG Last Name']
row_num = df[name].shape[0]

count = 0

for i in range(0, row_num):
    if test[i] == False:
        count = count + 1
if count != 0:
    print(f'{name} has a differences of {count}')

## 6. Loop through Rows and assign values based on Conditions

#### Function

#### Original Code

In [ ]:
df['Sheetname']['test'] = 'Nan'
test_frame = df['Sheetname']
for i in range(0,test_frame.shape[0]):
    if test_frame['From Number'][i] < test_frame['To Number'][i]: 
        test_frame['test'][i] = 'Check'
    else:
        test_frame['test'][i] = ''
df['Sheetname']['test'] = test_frame['test']

## 7. Reorder Columns by a Given List and Order

#### Function

#### Original Code

In [ ]:
#create a copy to check
OG_data = pd.read_excel('./'+ file + '/'+ file +' Data.xlsx', sheet_name = None)

In [ ]:
#get count of column header before start
counts_OG = dict()
for name, sheet in OG_data.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        counts_OG[name] = counts_OG.get(name, OG_data[name].shape[1])
counts_OG

In [ ]:
##Reorder first columns
col = ['col1', 'col2','col3']
col2 = ['col1', 'col2','col3']
temp = df
for name, sheet in df.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        df_list = df[name][col]
        df_j = temp[name].drop(col, axis=1)
        
        #join data horizontally
        df_i = pd.concat([df_list,df_j],axis=1)
        df[name] = df_i
        
        
    elif name == "Sheet1":
        df_list = df[name][col2]
        df_j = temp[name].drop(col2, axis=1)
        
        #join data horizontally
        df_i = pd.concat([df_list,df_j],axis=1)
        df[name] = df_i

In [ ]:
#Check if values match
for key, value in counts_OG.items():
    if counts_OG[key] != counts[key]:
        print(f"{key} have different value, OG: {counts_OG[key]} vs New: {counts[key]}")

In [ ]:
#Check if all columns in OG data are in the new one
for name, sheet in df.items():
    if name in ["Sheet1","Sheet2","Sheet3","Sheet4", "Sheet5","Sheet6"]:
        l = []
        for header in OG_data[name].columns:
            if header not in df[name].columns:
                l.append(header)
        if len(l) > 0:
            print(f"New {name} Tab is missing the following headers: {l}")

In [ ]:
#Push changes to Excel
with pd.ExcelWriter(data_path,engine="xlsxwriter") as writer:
    for name, sheet in df.items():
        df[name].to_excel(writer,sheet_name=name,index=False)
#writer.save()
writer.close()

## 8. Loop Through Rows and Make Each Cell Content Proper Case

#### Function